In [14]:
%matplotlib inline
from multiprocessing import Pool
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from IPython.display import Image, display
import time
from datetime import timedelta
import random
import sys

In [15]:
path_uid_to_cls = "imagenet_2012_challenge_label_map_proto.pbtxt"

In [16]:
data_dir = '../Inception/Inception_model/'


In [17]:
class Inception:
    tensor_name_input_jpeg = "DecodeJpeg/contents:0"
    tensor_name_input_image = "DecodeJpeg:0"
    tensor_name_resized_image = "ResizeBilinear:0"
    tensor_name_softmax = "softmax:0"
    tensor_name_softmax_logits = "softmax/logits:0"
    tensor_name_transfer_layer = "pool_3:0"
    tensor_name_transfer_conv_layer = 'mixed_10/join/concat_dim:0'
    tensor_name_transfer_convJoin_layer = "mixed_10/join:0"
    tensor_name_transfer_convMix_layer="mixed_10/tower_2/conv:0"

    def __init__(self ,data_dir):
        self.name_lookup = NameLookup( data_dir)
        self.graph = tf.Graph()
        with self.graph.as_default():
            path = os.path.join(data_dir, path_graph_def)
            with tf.gfile.FastGFile(path, 'rb') as file:
                graph_def = tf.GraphDef()
                graph_def.ParseFromString(file.read())
                tf.import_graph_def(graph_def, name='')
        self.y_pred             = self.graph.get_tensor_by_name(self.tensor_name_softmax)
        self.y_logits           = self.graph.get_tensor_by_name(self.tensor_name_softmax_logits)
        self.resized_image      = self.graph.get_tensor_by_name(self.tensor_name_resized_image)
        self.transfer_layer     = self.graph.get_tensor_by_name(self.tensor_name_transfer_layer)
        self.transfer_conv_layer = self.graph.get_tensor_by_name(self.tensor_name_transfer_conv_layer)
        self.transfer_convJoin_layer = self.graph.get_tensor_by_name(self.tensor_name_transfer_convJoin_layer)
        self.transfer_convMix_layer=self.graph.get_tensor_by_name("mixed_10/tower_2/conv:0")
        self.transfer_len = self.transfer_layer.get_shape()[3]
        self.transfer_conv_len = self.transfer_conv_layer.get_shape()
        
        self.session = tf.Session(graph=self.graph)
        
    def show_all_op(self):
        all_ops=self.session.graph.get_operations()
        for ele in all_ops:
            print ele.name
    def close(self):
        self.session.close()
    def _write_summary(self, logdir='summary/'):
        writer = tf.train.SummaryWriter(logdir=logdir, graph=self.graph)
        writer.close()
    def _create_feed_dict(self, image_path=None, image=None):
        if image is not None:
            feed_dict = {self.tensor_name_input_image: image}
        elif image_path is not None:
            image_data = tf.gfile.FastGFile(image_path, 'rb').read()
            feed_dict = {self.tensor_name_input_jpeg: image_data}
        else:
            raise ValueError("Either image or image_path must be set.")
        return feed_dict

    def classify(self, image_path=None, image=None):
        feed_dict = self._create_feed_dict(image_path=image_path, image=image)
        pred = self.session.run(self.y_pred, feed_dict=feed_dict)
        pred = np.squeeze(pred)
        return pred

    def get_resized_image(self, image_path=None, image=None):
        feed_dict = self._create_feed_dict(image_path=image_path, image=image)
        resized_image = self.session.run(self.resized_image, feed_dict=feed_dict)
        resized_image = resized_image.squeeze(axis=0)
        resized_image = resized_image.astype(float) / 255.0
        return resized_image

    def print_scores(self, pred, k=10, only_first_name=True):
        print 'print_score'
        idx = pred.argsort()
        print idx
        top_k = idx[-k:]
        print top_k
        for cls in reversed(top_k):
            name = self.name_lookup.cls_to_name(cls=cls, only_first_name=only_first_name)
            score = pred[cls]
            print name , score
            print("{0:>6.2%} : {1}".format(score, name))
    
    def transfer_values(self, image_path=None, image=None):
        feed_dict = self._create_feed_dict(image_path=image_path, image=image)
        transfer_values = self.session.run(self.transfer_layer , feed_dict=feed_dict)
        #transfer_values = np.squeeze(transfer_values)
        return transfer_values
    def transfer_conv_values(self , image_path = None , image = None):
        feed_dict = self._create_feed_dict(image_path = image_path , image = image)
        transfer_values = self.session.run(self.transfer_conv_layer , feed_dict=feed_dict)
        #transfer_values = np.squeeze(transfer_values)
        return transfer_values
    def transfer_convJoin_values(self , image_path = None , image = None):
        feed_dict = self._create_feed_dict(image_path = image_path , image = image)
        transfer_values = self.session.run(self.transfer_convJoin_layer , feed_dict=feed_dict)
        #transfer_values = np.squeeze(transfer_values)
        return transfer_values
    def transfer_convMix_values(self , image_path = None , image = None):
        feed_dict = self._create_feed_dict(image_path = image_path , image = image)
        transfer_values = self.session.run(self.transfer_convMix_layer , feed_dict=feed_dict)
        #transfer_values = np.squeeze(transfer_values)
        return transfer_values
#    def transfer_mixed10_tower1_conv1(self , image_path = None , image = None)#mixed_10/tower_1/conv_1
#        feed_dict = self._create_feed_dict(image_path = image_path , image = image)
#        transfer_values = self.session.run(self.transfer_convMix_layer , feed_dict=feed_dict)

def process_images(fn, images=None, image_paths=None):
        using_images = images is not None
        if using_images:
            num_images = len(images)
        else:
            num_images = len(image_paths)
        result = [None] * num_images
        for i in range(num_images):
            msg = "\r- Processing image: {0:>6} / {1}".format(i+1, num_images)
            sys.stdout.write(msg)
            sys.stdout.flush()
            if using_images:
                result[i] = fn(image=images[i])
            else:
                result[i] = fn(image_path=image_paths[i])
        print()
        result = np.array(result)
        return result
    
def transfer_values_cache(cache_path, model, images=None, image_paths=None):
        def fn():
            return process_images(fn=model.transfer_values, images=images, image_paths=image_paths)
        transfer_values = cache(cache_path=cache_path, fn=fn)
        return transfer_values
def transfer_conv_values_cache(cache_path, model, images=None, image_paths=None):
        def fn():
            return process_images(fn=model.transfer_conv_values, images=images, image_paths=image_paths)
        transfer_values = cache(cache_path=cache_path, fn=fn)
        return transfer_values
def transfer_convJoin_values_cache(cache_path, model, images=None, image_paths=None):
    def fn():
        return process_images(fn=model.transfer_convJoin_values, images=images, image_paths=image_paths)
    transfer_values = cache(cache_path=cache_path, fn=fn)
    return transfer_values
def transfer_convMix_values_cache(cache_path, model, images=None, image_paths=None):
    def fn():
        return process_images(fn=model.transfer_convMix_values, images=images, image_paths=image_paths)
    transfer_values = cache(cache_path=cache_path, fn=fn)
    return transfer_values
    

In [18]:
import sys
import os
import pickle
import numpy as np

def cache(cache_path, fn, *args, **kwargs):

    if os.path.exists(cache_path):
        # Load the cached data from the file.
        with open(cache_path, mode='rb') as file:
            obj = pickle.load(file)
        print("- Data loaded from cache-file: " + cache_path)
    else:
        obj = fn(*args, **kwargs)
        with open(cache_path, mode='wb') as file:
            pickle.dump(obj, file)
        print("- Data saved to cache-file: " + cache_path)
    return obj

def convert_numpy2pickle(in_path, out_path):
    data = np.load(in_path)
    with open(out_path, mode='wb') as file:
        pickle.dump(data, file)
        
        
if __name__ == '__main__':
    def expensive_function(a, b):
        return a * b
    print('Computing expensive_function() ...')
    result = cache(cache_path='cache_expensive_function.pkl',
                   fn=expensive_function, a=123, b=456)
    print('result =', result)
    print()
    class ExpensiveClass:
        def __init__(self, c, d):
            self.c = c
            self.d = d
            self.result = c * d

        def print_result(self):
            print('c =', self.c)
            print('d =', self.d)
            print('result = c * d =', self.result)

    print('Creating object from ExpensiveClass() ...')
    obj = cache(cache_path='cache_ExpensiveClass.pkl',
                fn=ExpensiveClass, c=123, d=456)
    obj.print_result()


Computing expensive_function() ...
- Data saved to cache-file: cache_expensive_function.pkl
('result =', 56088)
()
Creating object from ExpensiveClass() ...
- Data saved to cache-file: cache_ExpensiveClass.pkl
('c =', 123)
('d =', 456)
('result = c * d =', 56088)


In [19]:
tf.__version__

'0.12.1'

# Make PKL Global Average Pooling Data  

In [ ]:
def inceptionv3_pool3(src_folder , save_folder):
    """
    src extension : npy 
    """
    np_path=glob.glob(src_folder+'*.npy')
    pkl_name=path.replace('npy' , 'pkl').split('/')[-1]
    pkl_save_path=os.path.join(pkl_savefolder ,pkl_name)
    print pkl_save_path
    np_img=np.load(np_path)
    transfer_values_cache(pkl_save_path, images= np_img , model = model )


# Make PKL CONV 8X8 , 2048 Data  

#  Reduce Dimension

In [10]:
def reduce_dimension(conv , out_ch  , layer_name):
    in_ch=conv.get_shape()[-1]
    with tf.variable_scope(layer_name) as scope:
        w_conv=tf.get_variable(name = 'w_conv' , shape = [1,1,in_ch , out_ch] , initializer = tf.contrib.layers.xavier_initializer() ,dtype=tf.float32)
        b_conv=tf.Variable([out_ch] , dtype=tf.float32)
    layer=tf.nn.conv2d(conv , w_conv ,[1,2,2,1] , 'SAME') 
    layer=tf.add(layer  , b_conv)
    layer = tf.nn.relu(layer)
    return layer

In [11]:
def reduce_dimension_3x3(conv , out_ch  , layer_name):
    in_ch=conv.get_shape()[-1]
    with tf.variable_scope(layer_name) as scope:
        w_conv=tf.get_variable(name = 'w_conv' , shape = [3,3,in_ch , out_ch] , initializer = tf.contrib.layers.xavier_initializer() ,dtype=tf.float32)
        b_conv=tf.Variable([out_ch] , dtype=tf.float32)
    layer=tf.nn.conv2d(conv , w_conv ,[1,2,2,1] , 'SAME') 
    layer=tf.add(layer  , b_conv)
    layer = tf.nn.relu(layer)
    return layer

In [12]:
def reduce_dimension_5x5(conv , out_ch  , layer_name):
    in_ch=conv.get_shape()[-1]
    with tf.variable_scope(layer_name) as scope:
        w_conv=tf.get_variable(name = 'w_conv' , shape = [5,5,in_ch , out_ch] , initializer = tf.contrib.layers.xavier_initializer() ,dtype=tf.float32)
        b_conv=tf.Variable([out_ch] , dtype=tf.float32)
    layer=tf.nn.conv2d(conv , w_conv ,[1,2,2,1] , 'SAME') 
    layer=tf.add(layer  , b_conv)
    layer = tf.nn.relu(layer)
    return layer

#  Same Dimension

In [13]:
def same_dimension(conv , out_ch  , layer_name):
    in_ch=conv.get_shape()[-1]
    with tf.variable_scope(layer_name) as scope:
        w_conv=tf.get_variable(name = 'w_conv' , shape = [1,1,in_ch , out_ch] , initializer = tf.contrib.layers.xavier_initializer() ,dtype=tf.float32)
        b_conv=tf.Variable([out_ch] , dtype=tf.float32)
    layer=tf.nn.conv2d(conv , w_conv ,[1,1,1,1] , 'SAME') 
    layer=tf.add(layer  , b_conv)
    layer = tf.nn.relu(layer)
    return layer

In [14]:
def same_dimension_3x3(conv , out_ch  , layer_name):
    in_ch=conv.get_shape()[-1]
    with tf.variable_scope(layer_name) as scope:
        w_conv=tf.get_variable(name = 'w_conv' , shape = [3,3,in_ch , out_ch] , initializer = tf.contrib.layers.xavier_initializer() ,dtype=tf.float32)
        b_conv=tf.Variable([out_ch] , dtype=tf.float32)
    layer=tf.nn.conv2d(conv , w_conv ,[1,1,1,1] , 'SAME') 
    layer=tf.add(layer  , b_conv)
    layer = tf.nn.relu(layer)
    return layer

In [15]:
def same_dimension_5x5(conv , out_ch  , layer_name):
    in_ch=conv.get_shape()[-1]
    with tf.variable_scope(layer_name) as scope:
        w_conv=tf.get_variable(name = 'w_conv' , shape = [5,5,in_ch , out_ch] , initializer = tf.contrib.layers.xavier_initializer() ,dtype=tf.float32)
        b_conv=tf.Variable([out_ch] , dtype=tf.float32)
    layer=tf.nn.conv2d(conv , w_conv ,[1,1,1,1] , 'SAME') 
    layer=tf.add(layer  , b_conv)
    layer = tf.nn.relu(layer)
    return layer

In [16]:
def get_GAP(conv ,name, n_classes =2 ):
    n_ch=conv.get_shape()[-1]
    print conv.get_shape()
    gap = tf.reduce_mean(conv , axis = (1,2))
    init = tf.constant(np.random.rand(1,n_ch))
    with tf.variable_scope(name) as scope:
        gap_w=tf.get_variable('w' , shape = [ n_ch , n_classes])
        gap_b=tf.get_variable('b' ,  initializer=init)
        
    logits=tf.matmul(gap , gap_w)
    return logits

In [17]:
def fully_connect(flat_ ,out_ch , name):
    _ , in_ch=flat_.get_shape()
    with tf.variable_scope(name) as scope:
        fc_w=tf.get_variable('w' , shape=[in_ch,out_ch])
        fc_b=tf.get_variable('b'  , shape=[1,out_ch])
    layer=tf.matmul(flat_ , fc_w)
    layer=tf.add(layer , fc_b)
    return layer 

In [18]:
def get_class_map(label , conv , img_width):
    out_ch = int(conv.get_shape()[-1])
    conv_resized_bilinear= tf.image.resize_bilinear(conv , [ im_width , im_width]) # interpolation

    with tf.variable_scope('LeNet/GAP' , reuse = True):
        tranpose_w=tf.transpose(tf.get_variable('w')) # (2,   2048)
        _label_w = tf.gather(tf.transpose(tf.get_variable('w')) , label) # load tensor by name 'w'
        label_w = tf.reshape(_label_w , [out_ch ,1 ]) #(2048 ,1)
    conv_resized = tf.reshape(conv_resized_bilinear , [-1 , img_width*img_width , out_ch]) # 10000  , 2048
    classmap_batch_mul=tf.map_fn( lambda x: tf.matmul(x , label_w), conv_resized) # batchsize , 100,100 ,1
    #classmap_batch_mul = tf.batch_matmul(conv_resized, label_w)
    classmap = tf.reshape(classmap_batch_mul , [-1, img_width , img_width])
    return classmap , conv_resized, label_w , _label_w,tranpose_w,classmap_batch_mul,conv_resized_bilinear

In [20]:
def random_batch(x,y,batch_size):
    num_images = len(x)
    idx = np.random.choice(num_images , size=batch_size , replace = False)
    y_batch = y[idx]
    x_batch = x[idx]
    return x_batch , y_batch 

In [21]:
def pickle2numpy(path):
    f=open(path)
    imgs=pickle.load(f)
    return imgs  , path
def load_all_train_RAM(train_pkl_paths):
    row=8
    col=8
    n_ch=2048
    n_classes=2
    pool=Pool()
    label_folder_path = '/ssd/n_vs_ab/type1/1/'
    label_paths=crawl_folder(label_folder_path)
    n=len(train_pkl_paths)
    imgs_batch_list=[]
    labs_batch_list=[]
    count =0
    for imgs ,path in pool.imap( pickle2numpy, train_pkl_paths):
        name=path.split('/')[-1]
        labs_name = name.replace('img.pkl' , 'lab.npy')
        labs_path =os.path.join(label_folder_path , labs_name)
        labs=np.load(labs_path)
        imgs_batch_list.append(imgs)
        labs_batch_list.append(labs)
        msg = '\r-Progress : {0} / {1}'.format(str(count) , str(n))
        sys.stdout.write(msg)
        sys.stdout.flush()
        count+=1

    return imgs_batch_list , labs_batch_list

In [22]:
def choice_batch(imgs_batch_list , labs_batch_list):
    n_batch=len(imgs_batch_list)
    idx=random.randrange(0,n_batch)
   
    return imgs_batch_list[idx] , labs_batch_list[idx]

In [23]:
def load_train(train_pkl_paths):
    label_folder_path = '/ssd/n_vs_ab/type1/1/'
    #print train_pkl_paths
    idx=random.randrange(0,len(train_pkl_paths)-15)    
    train_pkl_path=train_pkl_paths[idx]
    name=train_pkl_path.split('/')[-1]
    lab_name = name.replace('img.pkl' , 'lab.npy')
    train_label_path =os.path.join(label_folder_path , lab_name)
    train_img=open(train_pkl_path , mode = 'rb') 
    train_img= pickle.load(train_img)
    train_lab= np.load(train_label_path)
    #print train_label_path
    #print train_pkl_path
    train_img=np.squeeze(train_img )
    return train_img , train_lab


In [24]:
def load_val(imgs_batch_list , labs_batch_list):
    batch_img_lab = zip(imgs_batch_list , labs_batch_list)
    train_imgs=[]
    train_labs=[]
    for img,lab in (batch_img_lab): 
        img=np.squeeze(img)
        print np.shape(img)
        train_imgs.append(list(img))
        train_labs.append(list(lab))
    shape=np.shape(train_imgs)
    print shape[0]
    print shape[1]
    train_imgs=np.reshape(train_imgs , newshape = (shape[0]*shape[1] ,shape[2],shape[3],-1))
    train_labs=np.reshape(train_labs , newshape = (shape[0]*shape[1] ,2))
    print np.shape(train_labs)
    print np.shape(train_imgs)
    #train_imgs =np.reshape(train_imgs,newshape=(15*80 , 2048))
    #train_labs =np.reshape(train_labs,newshape=(15*80 , n_classes))
    return train_imgs , train_labs

In [25]:
global_step = tf.Variable(initial_value=0 , name='global_step' , trainable=False)

In [19]:
def train_algo(y_conv,y_ , learning_rate):
    y_pred=tf.nn.softmax(y_conv)
    y_pred_cls = tf.argmax(y_pred , dimension=1)
    y_cls = tf.argmax(y_ , dimension =1)
    correct_prediction= tf.equal(y_pred_cls , y_cls)
    accuracy=tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
    cost  = tf.nn.softmax_cross_entropy_with_logits(y_conv , y_)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    return accuracy , y_pred , cost ,optimizer

# MODEL1

In [26]:
A=same_dimension(x_ , 1024 ,'A')
B=same_dimension(A , 512, 'B')
C=same_dimension(B , 256,'C')
flat_C = tf.contrib.layers.flatten(C)
y_conv=fully_connect(flat_C , 2 ,'fc1')
print y_conv

Tensor("Add_3:0", shape=(?, 2), dtype=float32)


In [27]:
accuracy,y_pred,cost,optimizer = train_algo(y_conv ,y_ , 0.001)

# MODEL2

In [28]:
A_1=same_dimension(x_ , 1024 ,'A_1')
B_1=reduce_dimension(A_1 , 512, 'B_1')
C_1=reduce_dimension(B_1 , 256,'C_1')
flat_C_1 = tf.contrib.layers.flatten(C_1)
y_conv_1=fully_connect(flat_C_1 , 2 ,'fc1_1')

In [29]:
accuracy_1 , y_pred_1 , cost_1,optimizer_1 = train_algo(y_conv_1 ,y_ , 0.001)

# Global_Step

In [30]:
global_step = tf.Variable(initial_value=0 , name='global_step' , trainable=False)

# Batch size

In [31]:
batch_size =32

# Load PKL PRE-Train image Data

In [32]:
all_paths=crawl_folder('/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl/')
train_pkl_paths =[]
count =0
for i,path in enumerate(all_paths):
    if 'train' in path and '.pkl' in path:
        train_pkl_paths.append(path)
        print path
        print count
        count+=1

/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_0_img.pkl
0
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_100_img.pkl
1
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_101_img.pkl
2
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_102_img.pkl
3
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_103_img.pkl
4
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_104_img.pkl
5
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_105_img.pkl
6
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_106_img.pkl
7
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_107_img.pkl
8
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_108_img.pkl
9
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_109_img.pkl
10
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_10_img.pkl
11
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_110_img.pkl
12
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_111_img.pkl
13
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_112_img.pkl
14
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pkl//train_113_img.pkl
15
/ssd/n_vs_ab/type1/1_conv_8x8_2048_pk

In [33]:
#train_pkl_paths

# # total batch Pkl Data

In [34]:
len(train_pkl_paths)

347

# Load All Train Data to RAM Memory

In [35]:
imgs_batch_list,labs_batch_list=load_all_train_RAM(train_pkl_paths)

-Progress : 346 / 347

# Load Validation Image

In [ ]:
val_img,val_lab=load_val(imgs_batch_list[:-15],labs_batch_list[:-15])

# Start Session

In [37]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


# Optimizing

In [45]:
def optimize(iteration):
    start_time = time.time()
    for i in range(iteration):
        #val_feed={x_: val_img , y_ : val_lab }
        train_img , train_lab=choice_batch(imgs_batch_list[:-15] , labs_batch_list[:-15])
        train_img=np.squeeze(train_img)
        x_batch , y_batch = random_batch(train_img,train_lab ,batch_size)
        feed_dict_train= {x_:x_batch , y_:y_batch}
        _ = sess.run([optimizer] ,  feed_dict=feed_dict_train)
        _ = sess.run([optimizer_1] ,  feed_dict=feed_dict_train)
        if (i %200 ==0 ) or (i ==iteration -1):
            
            batch_acc  , pred , loss = sess.run([accuracy , y_pred , cost ] , feed_dict = feed_dict_train)
            batch_acc_1  , pred_1 , loss_1 = sess.run([accuracy_1 , y_pred_1 , cost_1 ] , feed_dict = feed_dict_train)

            share= len(val_img) /batch_size
            remainder= len(val_img) %batch_size
            val_loss_mean=[]
            val_loss_mean_1=[]
            for i in range(share):
                feed_dict_val={x_: val_img[i:(i+1)*batch_size], y_:val_lab[i:(i+1)*batch_size]}
                val_acc  , va_pred , val_loss = sess.run([accuracy , y_pred , cost ] , feed_dict = feed_dict_val)
                val_acc_1  , va_pred_1 ,val_loss_1 = sess.run([accuracy_1 , y_pred_1 , cost_1 ] , feed_dict = feed_dict_val)
                val_loss_mean.append(np.asarray(val_loss))
                val_loss_mean_1.append(np.asarray(val_loss_1))
            feed_dict_val={x_: val_img[i:(i+1)*batch_size], y_:val_lab[i:(i+1)*batch_size]}
            val_acc  , va_pred , val_loss = sess.run([accuracy , y_pred , cost ] , feed_dict = feed_dict_val)
            val_acc_1  , va_pred_1 ,val_loss_1 = sess.run([accuracy_1 , y_pred_1 , cost_1 ] , feed_dict = feed_dict_val)
            val_loss_mean=train_loss_mean.mean()
            val_loss_mean_1=train_loss_mean_1,mean()
            msg = "Model 1 :Global Step : {0:>6} , Training Batch Accuracy: {1:>6.1%} , Training  Loss: {2}\n Validation Accuracy: {3:>6.1%} , Validation Loss:{4}"
            print (msg.format(i , batch_acc ,train_loss_mean , val_acc , val_loss_mean))
            msg = "Model 2 :Global Step : {0:>6} , Training Batch Accuracy: {1:>6.1%} , Training  Loss: {2}\n Validation Accuracy: {3:>6.1%} , Validation Loss:{4}"
            print (msg.format(i , batch_acc_1 ,train_loss_mean_1 , val_acc_1 , val_loss_mean_1))

    end_time = time.time()
    time_dif = end_time - start_time
    print ("Time usage:"+str(timedelta(seconds=int(round(time_dif)))))

In [ ]:
optimize(iteration=30000)

In [ ]:
def load_validated(train_pkl_paths):
    label_folder_path = '/ssd/n_vs_ab/type1/1/'
    pred_list=[]
    #print train_pkl_paths
    for idx in range(len(train_pkl_paths)-15 ,len(train_pkl_paths)) :
        print idx
        val_pkl_path=train_pkl_paths[idx]
        name=val_pkl_path.split('/')[-1]
        lab_name = name.replace('img.pkl' , 'lab.npy')
        val_pkl_path =os.path.join(label_folder_path , lab_name)
        val_img=open(val_pkl_path , mode = 'rb') 
        val_img= pickle.load(val_img)
        val_lab= np.load(val_label_path)
        feed_dict_train= {x_:val_img , y_:val_lab}
        pred = sess.run([y_pred] , feed_dict = feed_dict_train)
        pred_list.extend(pred)
    acc = pred_list.mean()
        #print train_label_path
        #print train_pkl_path
    return pred_list , acc 

In [ ]:
range(len(train_pkl_paths)-15 ,len(train_pkl_paths))